<a href="https://colab.research.google.com/github/Anurag38/NLP-Practicals/blob/main/Practical%206%20-%20Text%20Classification/Text_Classification_using_SVM_and_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
from gensim.models import FastText



In [2]:
import itertools
from gensim.models.word2vec import Text8Corpus
#from glove import Corpus, Glove
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
# from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
#from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
%matplotlib inline

In [3]:
# provide sql-like data manipulation tools. very handy.
import pandas as pd 
pd.options.mode.chained_assignment = None

# high dimensional vector computing library.
import numpy as np 
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim

# the word2vec model gensim class
from gensim.models.word2vec import Word2Vec 

# we'll talk about this down below
LabeledSentence = gensim.models.doc2vec.LabeledSentence 

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

# a tweet tokenizer from nltk.
from nltk.tokenize import TweetTokenizer 
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
train= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/NLP/Practical 6/taska.txt',header=None,sep='\t',names=['id','text','class'])
#valid_df= pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/AMI/dataset/agr_en_dev.csv',header=None,names=['id','text','class'])

#test_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/AMI/dataset/agr_en_fb_gold.csv',header=None,names=['id','text','class'])


In [6]:
train.info()
train.head()
train['class'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13240 entries, 0 to 13239
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      13240 non-null  int64 
 1   text    13240 non-null  object
 2   class   13240 non-null  object
dtypes: int64(1), object(2)
memory usage: 310.4+ KB


NOT    8840
OFF    4400
Name: class, dtype: int64

In [7]:
import re
def clean_text(comment):
    comment = comment.strip().strip('"')
    comment = comment.replace('_', ' ')
    comment = comment.replace("\\\\", "\\")
    comment = comment.replace('\\n', ' ')
    comment = comment.replace('\\n', ' ')
    comment = comment.lower()
    comment = re.sub(r'@[A-Za-z0-9]+','',comment)
    
    comment = re.sub('https?://[A-Za-z0-9./]+','',comment)
    commment = re.sub("[^a-zA-Z]", " ", comment)
    comment = re.sub(r'[^\w\s]','',comment)

    
    return comment

train['text'] = train['text'].fillna("something").values
train['ctext']=train['text'].apply(clean_text)
#train_df['ctext']=train_df['text']
#valid_df['ctext']=valid_df['text'].apply(clean_text)
#test_df['ctext']=test_df['text'].apply(clean_text)
train.head()


,id,text,class,ctext
0,86426,@USER She should ask a few native Americans wh...,OFF,she should ask a few native americans what th...
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,go home youre drunk maga trump2020 url
2,16820,Amazon is investigating Chinese employees who ...,NOT,amazon is investigating chinese employees who ...
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,someone shouldvetaken this piece of shit to a...
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,obama wanted liberals amp illegals to move i...


In [8]:
#tr=train_df[train_df['text']=='something']
#tr.head()

In [9]:
#train_df['text'] = train_df['text'].fillna("something").values
train['class'] = train['class'].map({'OFF': 0, 'NOT': 1})


In [10]:
train_df, test_df = train_test_split(train,test_size = 0.1)


In [11]:
import nltk
nltk.download('punkt')
train_df['tokenized_sents'] = train_df.apply(lambda row: nltk.word_tokenize(row['ctext']), axis=1)
test_df['tokenized_sents'] = test_df.apply(lambda row: nltk.word_tokenize(row['ctext']), axis=1)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
train_df.head()

,id,text,class,ctext,tokenized_sents
98,44209,@USER @USER what a baby! URL,1,what a baby url,"[what, a, baby, url]"
2213,63179,@USER how crazy youre a professor and you lie ...,1,how crazy youre a professor and you lie and s...,"[how, crazy, youre, a, professor, and, you, li..."
3548,41881,@USER Master of None was so fucking good.,0,master of none was so fucking good,"[master, of, none, was, so, fucking, good]"
4810,80502,@USER I can see how frustrating that is. Conse...,1,i can see how frustrating that is conservativ...,"[i, can, see, how, frustrating, that, is, cons..."
1351,65060,@USER @USER Then there should be no problem ha...,1,then there should be no problem having kavan...,"[then, there, should, be, no, problem, having,..."


In [13]:
x_train=train_df['tokenized_sents']
x_test=test_df['tokenized_sents']

y_train=train_df['class']
y_test=test_df['class']


In [14]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')

x_test = labelizeTweets(x_test, 'TEST')


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """
11916it [00:00, 201536.04it/s]
1324it [00:00, 6935.85it/s]


In [15]:
#tweet_w2v = FastText(size=400, min_count=1,sg=1)
tweet_w2v = Word2Vec(size=300, min_count=1,sg=1) #SKIPGRAM
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v.train([x.words for x in tqdm(x_train)],total_examples=len(x_train),epochs=10)


100%|██████████| 11916/11916 [00:00<00:00, 1595967.76it/s]


(1815059, 2330450)

In [16]:
tweet_w2v['hello']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.00314472, -0.13065872,  0.08991455, -0.12199759, -0.00978568,
       -0.04098298,  0.08693017, -0.06024295,  0.0075525 ,  0.07726957,
        0.07107034, -0.13799596,  0.05166671, -0.09340643,  0.06837691,
        0.03982214, -0.01618377, -0.12473425, -0.12859814, -0.03213874,
       -0.01839467,  0.04907308,  0.01367785,  0.04298516,  0.09651073,
        0.03812995,  0.13936377, -0.09867909,  0.09043328, -0.18033941,
        0.01383958, -0.20557696,  0.06169865,  0.01623362,  0.08468635,
       -0.1258927 ,  0.0059935 ,  0.0166661 , -0.01805684,  0.16648574,
       -0.06697562,  0.12380682,  0.01091013, -0.06491988, -0.10310332,
       -0.17081273,  0.04038997,  0.2096578 , -0.24963261,  0.04399724,
       -0.04198681,  0.0733701 ,  0.08029886, -0.0387301 , -0.04316225,
        0.17143124,  0.07347972,  0.0284666 , -0.05435783, -0.09613962,
       -0.08737934, -0.0416421 ,  0.06756239, -0.01392502,  0.02203156,
        0.03446636, -0.04230212, -0.03806321,  0.12149289,  0.00

In [ ]:
tweet_w2v['anurag']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


KeyError: ignored

In [17]:
#tweet_w2v.save('off-sg.vec')
#tweet_w2v.save('fb-en-w2vec-cbow.bin')
#tweet_w2v.wv.save_word2vec_format('fb-hi-w2vec-cbow.txt')

In [18]:
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(min_df=1)
#matrix = vectorizer.fit_transform(x_train)
matrix = vectorizer.fit_transform(train_df['ctext'])

tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

building tf-idf matrix ...
vocab size : 19483


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [19]:
#tweet_w2v['messi']

In [20]:
#tweet_w2v=FastText.load('fb-en-fasttext-sg.vec')


In [21]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [22]:
#tweet_w2v=Word2Vec.load('D:/ps/model/English/fasttext/coling-fasttext-eng-cbow.bin')
#from gensim.models.wrappers import FastText

#tweet_w2v= FastText.load_fasttext_format('D:/ps/model/English/fasttext/coling-fasttext-eng-cbow')

n_dim=300
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)



test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)



0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
11916it [00:02, 4180.34it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
1324it [00:00, 3892.83it/s]


###Shape
train_vecs_w2v.shape
test_vecs_w2v.shape
train_vecs_w2v[0]

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB



clf1 = LinearSVC()
clf2 = GaussianNB()

clf1.fit(train_vecs_w2v, y_train)
clf2.fit(train_vecs_w2v, y_train)

#print(clf.score(valid_vecs_w2v, y_valid))
print("SVM: ",clf1.score(test_vecs_w2v,y_test))
print("Naive Bayes: ",clf2.score(test_vecs_w2v,y_test))

SVM:  0.6774924471299094
Naive Bayes:  0.5317220543806647


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [24]:
from sklearn import metrics

y_pred_class = clf1.predict(test_vecs_w2v)

print(metrics.accuracy_score(y_test, y_pred_class))

print(metrics.confusion_matrix(y_test, y_pred_class))
print (metrics.precision_score(y_test, y_pred_class,average= 'weighted'))
print(metrics.recall_score(y_test, y_pred_class,average= 'weighted'))
print (metrics.f1_score(y_test, y_pred_class, average= 'weighted'))
#print (metrics.f1_score(y_test, y_pred_class))
print(metrics.classification_report(y_test, y_pred_class,target_names=['0','1']))


0.6774924471299094
[[141 272]
 [155 756]]
0.6545999294197039
0.6774924471299094
0.6606122768541524
              precision    recall  f1-score   support

           0       0.48      0.34      0.40       413
           1       0.74      0.83      0.78       911

    accuracy                           0.68      1324
   macro avg       0.61      0.59      0.59      1324
weighted avg       0.65      0.68      0.66      1324



In [25]:
from sklearn import metrics

y_pred_class = clf2.predict(test_vecs_w2v)

print(metrics.accuracy_score(y_test, y_pred_class))

print(metrics.confusion_matrix(y_test, y_pred_class))
print (metrics.precision_score(y_test, y_pred_class,average= 'weighted'))
print(metrics.recall_score(y_test, y_pred_class,average= 'weighted'))
print (metrics.f1_score(y_test, y_pred_class, average= 'weighted'))
#print (metrics.f1_score(y_test, y_pred_class))
print(metrics.classification_report(y_test, y_pred_class,target_names=['0','1']))


0.5317220543806647
[[322  91]
 [529 382]]
0.6737189467464182
0.5317220543806647
0.5387567595617556
              precision    recall  f1-score   support

           0       0.38      0.78      0.51       413
           1       0.81      0.42      0.55       911

    accuracy                           0.53      1324
   macro avg       0.59      0.60      0.53      1324
weighted avg       0.67      0.53      0.54      1324

